In [8]:
import pandas as pd
import numpy as np
from category_encoders.one_hot import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from category_encoders.leave_one_out import LeaveOneOutEncoder
import re
import warnings
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import matthews_corrcoef
import xgboost as xgb

warnings.simplefilter("ignore")
%matplotlib inline

In [3]:
dataset = pd.read_csv("train.csv")

In [4]:
def cleaning(df):

    threshold = 100

    cat_feats = ["cap-shape","cap-surface","cap-color","does-bruise-or-bleed","gill-attachment",
                 "gill-spacing","gill-color","stem-root","stem-surface","stem-color","veil-type",
                 "veil-color","has-ring","ring-type","spore-print-color","spore-print-color",
                 "habitat","season"]

    for feat in cat_feats:
        df[feat] = df[feat].fillna('missing')
        #df.loc[df[feat].value_counts(dropna=False)[df[feat]].values < threshold, feat] = "noise"
        df[feat] = df[feat].astype('category')

    return df

dataset  = cleaning(dataset)

In [11]:
X = dataset
X = X.drop(["class"], axis="columns")
y = dataset["class"]
y = y.map({"p":0, "e":1})

In [54]:
clf = xgb.XGBClassifier(tree_method="hist", 
                        objective ='binary:hinge' , 
                        enable_categorical=True, 
                        #min_sample = 5,
                        #max_sample = 15,
                        max_bin=128,
                        learning_rate= 0.1,
                        colsample_bytree=0.6,
                        n_estimator = 500,
                        device="cuda")

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

In [61]:
clf.fit(X_train, y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.6, device='cuda', early_stopping_rounds=None,
              enable_categorical=True, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=128,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimator=500, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, ...)

In [57]:
y_test_pred = clf.predict(X_test)

In [58]:
print(f"Decision tree accuracy train: {accuracy_score(y_test_pred, y_test):.8f}")

ValueError: Found input variables with inconsistent numbers of samples: [2077964, 623389]

In [59]:
print(round(matthews_corrcoef(y_test_pred, y_test),5))

ValueError: Found input variables with inconsistent numbers of samples: [2077964, 623389]

In [46]:
test = pd.read_csv("test.csv")
test  = cleaning(test)

X_test = test

In [47]:
y_pred = clf.predict(X_test)

In [48]:
pre_kaggle = pd.read_csv('predict_kaggle.csv')

In [49]:
pre_kaggle_0 = pre_kaggle['0']

In [50]:
pre_kaggle_0 = pre_kaggle_0.replace('p', 0)

In [51]:
pre_kaggle_0 = pre_kaggle_0.replace('e', 1)

In [52]:
print(f"Decision tree accuracy train: {accuracy_score(y_pred,pre_kaggle_0):.8f}")

Decision tree accuracy train: 0.46349889


In [53]:
print(round(matthews_corrcoef(y_pred,pre_kaggle_0),5))

0.07402
